In [1]:
import controllables.energyplus as _ooep_
from energyplus.dataset.basic import dataset as _epds_

simulator = _ooep_.World(
    input=_ooep_.World.Specs.Input(
        world='tmp_timestep 10 min.idf',
        weather='SGP_Singapore_486980_IWEC.epw',
    ),
    # output=_ooep_.World.Specs.Output(
    #     report='./tmp',
    # ),
    runtime=_ooep_.World.Specs.Runtime(
        recurring=True,
        #design_day=True,
    ),
)

# add progress provider
_ = simulator.add('logging:progress')

  0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
_ = simulator.awaitable.run()

/usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_hlvbl9og'>
  _warnings.warn(warn_message, ResourceWarning)
/usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_jw8xvbgo'>
  _warnings.warn(warn_message, ResourceWarning)
/usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_puimws66'>
  _warnings.warn(warn_message, ResourceWarning)
/usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_1lbwywfy'>
  _warnings.warn(warn_message, ResourceWarning)
/usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_jh3lrm6z'>
  _warnings.warn(warn_message, ResourceWarning)
/usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDi

In [ ]:
import numpy as _numpy_

from controllables.core import TemporaryUnavailableError
from controllables.core.tools.gymnasium import BoxSpace, DictSpace
from controllables.core.tools.rllib import Env
from controllables.energyplus import Actuator, OutputVariable
from controllables.energyplus import System


class UserEnv(Env):
    action_space = DictSpace({
        'thermostat': BoxSpace(
            low=15., high=20.,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(
            Actuator.Ref(
                type='Zone Temperature Control',
                control_type='Heating Setpoint',
                key='MAIN ZONE',
            )            
        )
    })

    observation_space = DictSpace({
        'temperature': BoxSpace(
            low=-_numpy_.inf, high=+_numpy_.inf,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(
            OutputVariable.Ref(
                type='Zone Mean Air Temperature',
                key='MAIN ZONE',
            )
        ),
    })

    @staticmethod
    def reward(agent):
        r"""
        Reward function.

        This reward function aims to minimize the control error, 
        i.e., the difference between the thermostat setpoint and the actual temperature.
        """
        
        try:
            return -abs(
                agent.observation['temperature'].value - agent.action['thermostat'].value
            )
        except TemporaryUnavailableError:
            return 0.

    def __init__(self, config: dict = dict()):
        super().__init__({
            'action_space': self.__class__.action_space,
            'observation_space': self.__class__.observation_space,
            'reward': self.__class__.reward,
            **config,
        })

    def run(self):
        system = System(
            building='all_room_have_hvac.idf',
            weather='SGP_Singapore_486980_IWEC.epw',
            # TODO
            report='tmp/',
            repeat=True,
        )
        # system.add('logging:progress')
        self.__attach__(system).schedule_episode()
        system.start().wait()

In [3]:
import numpy as _numpy_
import gymnasium as _gymnasium_
import pandas as pd
import os

from controllables.core.tools.gymnasium import (
    BoxSpace,
    DictSpace,
)
from controllables.core.tools.ray import (
    ExternalEnv,
)

import ray
from ray import tune , air
from ray.tune.schedulers import PopulationBasedTraining

from controllables.energyplus import (
    World,
    Actuator,
    OutputVariable,
)
import pythermalcomfort as pytc
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.impala import ImpalaConfig
from ray.rllib.algorithms.sac import SACConfig

from ray.rllib.algorithms.callbacks import DefaultCallbacks

import logging
import csv
import math

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', filename='training.log')

class CustomExternalEnv(ExternalEnv):
    def __init__(self, env_config):
        pass

    def step(self, action):
        pass

class RewardFunction:
    def __init__(self, metab_rate=1.5, clothing=.5, pmv_limit=.5):
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
    
   
    def __call__(self, agent):
        observation = agent.observation.value     
        AHU_COOLING_COIL = observation['AHU COOLING COIL']
        #Fan_Electricity_Rate = observation['Fan Electricity Rate']
        Office_Occupancy = observation['Office Occupancy']
        total_reward = 0
        zones = ['1FWEST', '1FEAST', '0FWEST', '0FEAST']
        for zone in zones:
            tdb = observation[f'temperature:drybulb_{zone}']
            tr = observation[f'temperature:radiant_{zone}']
            rh = observation[f'humidity_{zone}']
            vr = pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate)
            clo = pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate)
            pmv = pytc.models.pmv_ppd(
                    tdb=tdb, 
                    tr=tr, 
                    vr=vr, 
                    rh=rh, 
                    met=self._metab_rate, 
                    clo=clo,
                    limit_inputs=False,
                )['pmv']
            zone_reward = 2 * Office_Occupancy * ((self._pmv_limit - _numpy_.abs(pmv)) / self._pmv_limit)
            total_reward += zone_reward
        total_reward -= (AHU_COOLING_COIL / 180000)
        # inputs = {'AHU_energy': AHU_COOLING_COIL, 'tdb': observation['temperature:drybulb'],
        #         'tr': observation['temperature:radiant'], 
        #         'vr': pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
        #         'rh': observation['humidity'], 'met': self._metab_rate,
        #         'clo': pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
        #         'pmv':pmv ,
        #         'reward':reward}
        # for name, value in inputs.items():
        #     if math.isnan(value):
        #         print(f"NaN detected in input: {name}")
        #         print(dict(
        #             tdb=(observation['temperature:drybulb']), 
        #             tr=observation['temperature:radiant'], 
        #             # calculate relative air speed
        #             vr=pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
        #             rh=observation['humidity'], 
        #             met=self._metab_rate, 
        #             # calculate dynamic clothing
        #             clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
        #         ))
        #         raise Exception('TODO')
        # if math.isnan(reward):
        #     reward = 0


        return total_reward



from ray.rllib.algorithms.callbacks import DefaultCallbacks


class TraceCallbacks(DefaultCallbacks):
    def on_episode_start(self, *, worker, episode, base_env, **kwargs) -> None:
        from energyplus.ooep.specs.tools import VariableHistory

        env = worker.env

        episode._user_history = VariableHistory()
        display(
            episode._user_history.plot({
                'traces': [{
                    'x': env.world['wallclock:calendar'], 
                    'y': env.reward,
                }],
            }, autoupdate=1_000)
        )

    def on_episode_step(self, *, worker, episode, base_env, **kwargs) -> None:
        episode._user_history.poll()


config = (
    PPOConfig()
    .environment(
        ExternalEnv,
        env_config=ExternalEnv.Config(
            action_space=DictSpace({
                'thermostat_1FWEST': BoxSpace(
                    low=22., high=30.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Zone Temperature Control',
                    control_type='Cooling Setpoint',
                    key='1FFIRSTFLOORWEST:OPENOFFICE',
                )),
                'thermostat_1FEAST': BoxSpace(
                    low=22., high=30.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Zone Temperature Control',
                    control_type='Cooling Setpoint',
                    key='1FFIRSTFLOOREAST:OPENOFFICE',
                )),
                'thermostat_0FWEST': BoxSpace(
                    low=22., high=30.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Zone Temperature Control',
                    control_type='Cooling Setpoint',
                    key='0FGROUNDFLOORWEST:OPENOFFICE',
                )),
                'thermostat_0FEAST': BoxSpace(
                    low=22., high=30.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Zone Temperature Control',
                    control_type='Cooling Setpoint',
                    key='0FGROUNDFLOOREAST:OPENOFFICE',
                )),
            }),    
            observation_space=DictSpace({
                #1FWEST
                'temperature:drybulb_1FWEST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Air Temperature',
                    key='1FFIRSTFLOORWEST:OPENOFFICE',
                )),
                'temperature:radiant_1FWEST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Radiant Temperature',
                    key='1FFIRSTFLOORWEST:OPENOFFICE',
                )),
                'humidity_1FWEST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Air Relative Humidity',
                    key='1FFIRSTFLOORWEST:OPENOFFICE',
                )),

                #1FEAST
                'temperature:drybulb_1FEAST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Air Temperature',
                    key='1FFIRSTFLOOREAST:OPENOFFICE',
                )),
                'temperature:radiant_1FEAST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Radiant Temperature',
                    key='1FFIRSTFLOOREAST:OPENOFFICE',
                )),
                'humidity_1FEAST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Air Relative Humidity',
                    key='1FFIRSTFLOOREAST:OPENOFFICE',
                )),

                #0FWEST
                'temperature:drybulb_0FWEST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Air Temperature',
                    key='0FGROUNDFLOORWEST:OPENOFFICE',
                )),
                'temperature:radiant_0FWEST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Radiant Temperature',
                    key='0FGROUNDFLOORWEST:OPENOFFICE',
                )),
                'humidity_0FWEST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Air Relative Humidity',
                    key='0FGROUNDFLOORWEST:OPENOFFICE',
                )),

                #0FEAST
                'temperature:drybulb_0FEAST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Air Temperature',
                    key='0FGROUNDFLOOREAST:OPENOFFICE',
                )),
                'temperature:radiant_0FEAST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Radiant Temperature',
                    key='0FGROUNDFLOOREAST:OPENOFFICE',
                )),
                'humidity_0FEAST': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Air Relative Humidity',
                    key='0FGROUNDFLOOREAST:OPENOFFICE',
                )),

                #AHU
                'AHU COOLING COIL': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Cooling Coil Total Cooling Rate',
                    key='AIR LOOP AHU COOLING COIL',
                )),
                'Fan Electricity Rate': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Fan Electricity Rate',
                    key='AIR LOOP AHU SUPPLY FAN',
                )),
                'Office Occupancy':BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Schedule Value',
                    key='Office_OpenOff_Occ',
                )),
            }),
            reward_function=RewardFunction(),
            episode_events={
                'step': 'begin_zone_timestep_after_init_heat_balance',
            },
            system=lambda: simulator
        ),  

    )
    .rollouts(
        num_rollout_workers=0,
        enable_connectors=False,
    )
    .training(
        model={"fcnet_hiddens": [128, 128]},
        lr=0.0001,
        train_batch_size=1000,
    )
    .framework("torch")    
)
algo = config.build()

/usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_k8sl442p'>
  _warnings.warn(warn_message, ResourceWarning)
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:442: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.


In [5]:
#%pip install -U ../../EnergyPlus-OOEP/

In [4]:
def train():
    global algo
    for _ in range(200):
        print(algo.train())

import asyncio
async def run_train():
    asyncio.get_running_loop().run_in_executor(None, train)

await asyncio.create_task(run_train())

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 3.567764001233237, 'cur_kl_coeff': 0.2, 'cur_lr': 0.00010000000000000002, 'total_loss': 9.59916451772054, 'policy_loss': -0.06418245832125345, 'vf_loss': 9.661329137711299, 'vf_explained_var': -0.0008304505121140253, 'kl': 0.010089638425935327, 'entropy': 5.627097879137311, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 105.5, 'diff_num_grad_updates_vs_sampler_policy': 104.5}}, 'num_env_steps_sampled': 1000, 'num_env_steps_trained': 1000, 'num_agent_steps_sampled': 1000, 'num_agent_steps_trained': 1000}, 'sampler_results': {'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hi

In [8]:
algo.workers.local_worker().env.action.value

{'thermostat_1FWEST': array(27.501886, dtype=float32),
 'thermostat_1FEAST': array(22., dtype=float32)}

In [5]:
c

# varlogger.plot({
#     'traces': [
#         dict(
#             x='wallclock:calendar', 
#             y=Actuator.Ref(
#                 type='Fan',
#                 control_type='Fan Air Mass Flow Rate',
#                 key='AIR LOOP AHU SUPPLY FAN',
#             ),
#         ),        
#     ],
# })


In [6]:
# varlogger.track('clock',
#     'wallclock:calendar'
# )

varlogger.track('Fan Air Mass Flow Rate', Actuator.Ref(
    type='Fan',
    control_type='Fan Air Mass Flow Rate',
    key='AIR LOOP AHU SUPPLY FAN',
),
)

varlogger.track('thermostat_1FEAST',Actuator.Ref(
    type='Schedule:Compact',
    control_type='Schedule Value',
    key='1FFIRSTFLOOREAST:OPENOFFICE',
)
)

varlogger.track('thermostat_1FWEST',Actuator.Ref(
    type='Schedule:Compact',
    control_type='Schedule Value',
    key='1FFIRSTFLOORWEST:OPENOFFICE COOLING SETPOINT SCHEDULE',
)
)

varlogger.track('Zone Mean Air Temperature',OutputVariable.Ref(
    type='Zone Mean Air Temperature',
    key='1FFIRSTFLOORWEST:OPENOFFICE',
)
)

varlogger.track(OutputVariable.Ref(
    type='Zone Air Relative Humidity',
    key='1FFIRSTFLOORWEST:OPENOFFICE',
)
)

varlogger.track('Zone Mean Radiant Temperature',OutputVariable.Ref(
    type='Zone Mean Radiant Temperature',
    key='1FFIRSTFLOORWEST:OPENOFFICE',
)
)

varlogger.track('Cooling Coil Total Cooling Rate',OutputVariable.Ref(
    type='Cooling Coil Total Cooling Rate',
    key='AIR LOOP AHU COOLING COIL',
)
)

varlogger.track('Fan Electricity Rate',OutputVariable.Ref(
    type='Fan Electricity Rate',
    key='AIR LOOP AHU SUPPLY FAN',
)
)

varlogger.track('Office Occupancy',OutputVariable.Ref(
    type='Schedule Value',
    key='Office_OpenOff_Occ',
)
)


In [8]:
varlogger._data

{'Fan Air Mass Flow Rate': History.Record(ref=Actuator.Ref(type='Fan', control_type='Fan Air Mass Flow Rate', key='AIR LOOP AHU SUPPLY FAN'), values=deque([0.0], maxlen=10000)),
 'thermostat_1FEAST': History.Record(ref=Actuator.Ref(type='Schedule:Compact', control_type='Schedule Value', key='1FFIRSTFLOOREAST:OPENOFFICE'), values=deque([], maxlen=10000)),
 'thermostat_1FWEST': History.Record(ref=Actuator.Ref(type='Schedule:Compact', control_type='Schedule Value', key='1FFIRSTFLOORWEST:OPENOFFICE COOLING SETPOINT SCHEDULE'), values=deque([], maxlen=10000)),
 'Zone Mean Air Temperature': History.Record(ref=OutputVariable.Ref(type='Zone Mean Air Temperature', key='1FFIRSTFLOORWEST:OPENOFFICE'), values=deque([], maxlen=10000)),
 OutputVariable.Ref(type='Zone Air Relative Humidity', key='1FFIRSTFLOORWEST:OPENOFFICE'): History.Record(ref=OutputVariable.Ref(type='Zone Air Relative Humidity', key='1FFIRSTFLOORWEST:OPENOFFICE'), values=deque([], maxlen=10000)),
 'Zone Mean Radiant Temperature': 

In [10]:
varlogger.plot({
    'traces': [
        dict(
            x='clock', 
            y='Fan Air Mass Flow Rate',
        ),
        dict(
            x='clock', 
            y='thermostat_1FEAST',
        ),
        dict(
            x='clock', 
            y='thermostat_1FWEST',
        ),
        dict(
            x='clock', 
            y='Zone Mean Air Temperature',
        ),
        dict(
            x='clock', 
            y='Zone Mean Radiant Temperature',
        ),
        dict(
            x='clock', 
            y='Cooling Coil Total Cooling Rate',
        ),
        dict(
            x='clock', 
            y='Fan Electricity Rate',
        ),
        dict(
            x='clock', 
            y='Office Occupancy',
        ),
    ],
}, autoupdate=1_000)


In [7]:
import itables as _itables_
df = varlogger.dataframe()
_itables_.show(df)
# df.to_csv('datasave/data.csv', index=False, sep=';')


ValueError: All arrays must be of the same length